In [1]:
from graphdatascience import GraphDataScience
from torch_geometric.data import Data, download_url
import torch

In [2]:
gds = GraphDataScience("bolt://localhost:7687", auth=('neo4j', 'neo4jneo4j'), database="fb15k-237")

In [3]:
url = ('https://raw.githubusercontent.com/villmow/datasets_knowledge_embedding/master/FB15k-237')
raw_file_names = ['train.txt', 'valid.txt', 'test.txt']
raw_dir = './data_from_url'
for filename in raw_file_names:
    download_url(f'{url}/{filename}', raw_dir)

Using existing file train.txt
Using existing file valid.txt
Using existing file test.txt


In [4]:
def process():
    data_list_, node_dict_, rel_dict_ = [], {}, {}
    for file_name in raw_file_names:
        file_name_path = raw_dir + '/' + file_name
        with open(file_name_path, 'r') as f:
            data = [x.split('\t') for x in f.read().split('\n')[:-1]]

        edge_index = torch.empty((2, len(data)), dtype=torch.long)
        edge_type = torch.empty(len(data), dtype=torch.long)
        for i, (src, rel, dst) in enumerate(data):
            if src not in node_dict_:
                node_dict_[src] = len(node_dict_)
            if dst not in node_dict_:
                node_dict_[dst] = len(node_dict_)
            if rel not in rel_dict_:
                rel_dict_[rel] = len(rel_dict_)

            edge_index[0, i] = node_dict_[src]
            edge_index[1, i] = node_dict_[dst]
            edge_type[i] = rel_dict_[rel]

        data = Data(edge_index=edge_index, edge_type=edge_type)
        data_list_.append(data)

    for data in data_list_:
        data.num_nodes = len(node_dict_)

    return data_list_, node_dict_, rel_dict_

data_list, node_dict, rel_dict = process()

In [5]:
gds.run_cypher("CREATE CONSTRAINT entity_id FOR (e:Entity) REQUIRE e.id IS UNIQUE")

ClientError: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent constraint already exists, 'Constraint( id=4, name='entity_id', type='UNIQUENESS', schema=(:Entity {id}), ownedIndex=3 )'.}

In [6]:
rel_id_to_text_dict = {}
for k in rel_dict:
    text = k
    id = rel_dict[k]
    rel_id_to_text_dict[id] = text

In [7]:
print(data_list)
print(data_list[0].edge_index[0][1].item())
print(data_list[0].edge_type)

[Data(edge_index=[2, 272115], edge_type=[272115], num_nodes=14541), Data(edge_index=[2, 17535], edge_type=[17535], num_nodes=14541), Data(edge_index=[2, 20466], edge_type=[20466], num_nodes=14541)]
2
tensor([  0,   1,   2,  ..., 170,  30,  38])


In [8]:
def write_chunk(chunk_dict):
    gds.run_cypher(
            "UNWIND $nodes AS node CREATE (n:Entity {id: node[1], value: node[0]})",
            params={"nodes": list(chunk_dict.items())},
        )
    print(f"Written {len(chunk_dict)} elements...")

idx = 0
chunk_size = 1000
chunk_dict = {}
for k in node_dict:
    chunk_dict[k] = node_dict[k]
    idx += 1
    if idx % chunk_size == 0:
        write_chunk(chunk_dict)
        chunk_dict = {}
if len(chunk_dict) > 0:
    write_chunk(chunk_dict)
print(f"TOTAL records: {idx} from {len(node_dict)}")

Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 541 elements...
TOTAL records: 14541 from 14541


In [9]:
train_data = data_list[0]
val_data = data_list[1]
test_data = data_list[2]

In [10]:
def write_rel_chunk(ll:list, label):
    gds.run_cypher(
            "UNWIND $list AS l MATCH (e_s:Entity {id: l[0]}), (e_t:Entity {id: l[1]}) "+
            "CREATE (e_s)-["+label+" { rel_id: l[2], text: l[3] }]->(e_t)",
            params={"list": ll},
        )
    print(f"Written {len(ll)} elements...")


def create_rels(data:Data, label:str):
    idx = 0
    chunk_size = 1000
    chunk_list = []
    print("Writing " + label + " relationships")
    for i in range(data.num_edges):
        source = data.edge_index[0, i].item()
        target = data.edge_index[1, i].item()
        id = data.edge_type[i].item()
        text = rel_id_to_text_dict[id]
        l = [source, target, id, text]
        chunk_list.append(l)
        idx += 1
        if idx % chunk_size == 0:
            write_rel_chunk(chunk_list, label)
            chunk_list = []
    if len(chunk_list) > 0:
        write_rel_chunk(chunk_list, label)
    print(f"TOTAL records: {idx} from {data.num_edges}")

create_rels(test_data, ":TEST")
create_rels(val_data, ":VAL")
create_rels(train_data, ":TRAIN")

Writing :TEST relationships
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 466 elements...
TOTAL records: 20466 from 20466
Writing :VAL relationships
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 e

In [11]:
# Node: (:Entity {id:int, value:str})
# Edge: [:(TRAIN|TEST|VAL) {rel_id:int, text:str}]

In [13]:
def get_data_from_db(edge_label):
    node_projection = {"Entity": {"properties": "id"}}
    relationship_projection = {edge_label : {"orientation": "NATURAL", "properties": "rel_id"}}
    G, result = gds.graph.project(
        "fb15k-graph-t"+edge_label,
        node_projection,
        relationship_projection,
    )
    print(f"The projection took {result['projectMillis']} ms")

    # We can use convenience methods on `G` to check if the projection looks correct
    print(f"Graph '{G.name()}' node count: {G.node_count()}")
    print(f"Graph '{G.name()}' node labels: {G.node_labels()}")
    print(f"Graph '{G.name()}' relationship count: {G.relationship_count()}")

    return G

def get_whole_dataset():
    node_projection = {"Entity": {"properties": "id"}}
    relationship_projection = {
        "TRAIN" : {"orientation": "NATURAL", "properties": "rel_id"},
        "TEST" : {"orientation": "NATURAL", "properties": "rel_id"},
        "VAL" : {"orientation": "NATURAL", "properties": "rel_id"},
    }
    G, result = gds.graph.project(
        "fb15k-graph-whole",
        node_projection,
        relationship_projection,
    )
    print(f"The projection took {result['projectMillis']} ms")

    # We can use convenience methods on `G` to check if the projection looks correct
    print(f"Graph '{G.name()}' node count: {G.node_count()}")
    print(f"Graph '{G.name()}' node labels: {G.node_labels()}")
    print(f"Graph '{G.name()}' relationship count: {G.relationship_count()}")

    return G

In [14]:
train_db_data_G = get_data_from_db("TRAIN")
test_db_data_G = get_data_from_db("TEST")
val_db_data_G = get_data_from_db("VAL")
db_data_G = get_whole_dataset()

The projection took 47 ms
Graph 'fb15k-graph-tTRAIN' node count: 14541
Graph 'fb15k-graph-tTRAIN' node labels: ['Entity']
Graph 'fb15k-graph-tTRAIN' relationship count: 272115
The projection took 9 ms
Graph 'fb15k-graph-tTEST' node count: 14541
Graph 'fb15k-graph-tTEST' node labels: ['Entity']
Graph 'fb15k-graph-tTEST' relationship count: 20466
The projection took 22 ms
Graph 'fb15k-graph-tVAL' node count: 14541
Graph 'fb15k-graph-tVAL' node labels: ['Entity']
Graph 'fb15k-graph-tVAL' relationship count: 17535
The projection took 51 ms
Graph 'fb15k-graph-whole' node count: 14541
Graph 'fb15k-graph-whole' node labels: ['Entity']
Graph 'fb15k-graph-whole' relationship count: 310116


In [ ]:
# gds.graph.drop(train_db_data_G)
# gds.graph.drop(test_db_data_G)
# gds.graph.drop(val_db_data_G)

In [15]:
print(db_data_G)

Graph(name=fb15k-graph-whole, node_count=14541, relationship_count=310116)


In [23]:
node_properties = gds.graph.nodeProperties.stream(
    db_data_G,
    ["id"],
    separate_property_columns=True,
)
print(node_properties)

       nodeId    id
0       10000  9695
1       10001  9696
2       10002  9697
3       10003  9698
4       10004  9699
...       ...   ...
14536    9995  9690
14537    9996  9691
14538    9997  9692
14539    9998  9693
14540    9999  9694

[14541 rows x 2 columns]


In [24]:
nodeId_to_id = dict(zip(node_properties.nodeId, node_properties.id))

In [25]:
sample_topology_df = gds.beta.graph.relationships.stream(db_data_G)
# Let's see what we got:
display(sample_topology_df)

,sourceNodeId,targetNodeId,relationshipType
0,0,811,VAL
1,0,2664,VAL
2,0,4671,VAL
3,0,9126,VAL
4,0,2,TRAIN
...,...,...,...
310111,9999,3249,TRAIN
310112,9999,5257,TRAIN
310113,9999,6475,TRAIN
310114,9999,11890,TRAIN


In [28]:
rels_tmp = gds.graph.relationshipProperties.stream(db_data_G, ["rel_id"], separate_property_columns=True)
display(rels_tmp)
rels_tmp.rel_id.astype(int)
display(rels_tmp.sourceNodeId.map(lambda x: nodeId_to_id[x]))

,sourceNodeId,targetNodeId,relationshipType,rel_id
0,0,811,VAL,118.0
1,0,2664,VAL,195.0
2,0,4671,VAL,0.0
3,0,9126,VAL,23.0
4,0,2,TRAIN,0.0
...,...,...,...,...
310111,9999,3249,TRAIN,15.0
310112,9999,5257,TRAIN,154.0
310113,9999,6475,TRAIN,94.0
310114,9999,11890,TRAIN,62.0


KeyboardInterrupt: 

In [33]:
topology = [rels_tmp.sourceNodeId.map(lambda x: nodeId_to_id[x]), rels_tmp.targetNodeId.map(lambda x: nodeId_to_id[x])]
edge_index = torch.tensor(topology, dtype=torch.long)
edge_type = torch.tensor(rels_tmp.rel_id.astype(int), dtype=torch.long)
display(edge_index)
display(edge_type)
data = Data(edge_index=edge_index, edge_type=edge_type)
data.num_nodes = len(nodeId_to_id)
print(data)

tensor([[    0,     0,     0,  ...,  9694,  9694,  9694],
        [  774,  2550,  4494,  ...,  6225, 11585, 11585]])

tensor([118, 195,   0,  ...,  94,  62, 127])

Data(edge_index=[2, 310116], edge_type=[310116], num_nodes=14541)


In [34]:
def create_tensor(graph):
    rels_tmp = gds.graph.relationshipProperties.stream(graph, ["rel_id"], separate_property_columns=True)
    topology = [rels_tmp.sourceNodeId.map(lambda x: nodeId_to_id[x]), rels_tmp.targetNodeId.map(lambda x: nodeId_to_id[x])]
    edge_index = torch.tensor(topology, dtype=torch.long)
    edge_type = torch.tensor(rels_tmp.rel_id.astype(int), dtype=torch.long)
    data = Data(edge_index=edge_index, edge_type=edge_type)
    data.num_nodes = len(nodeId_to_id)
    display(data)
    return data

train_tensor = create_tensor(train_db_data_G)
test_tensor = create_tensor(test_db_data_G)
val_tensor = create_tensor(val_db_data_G)

Data(edge_index=[2, 272115], edge_type=[272115], num_nodes=14541)

Data(edge_index=[2, 20466], edge_type=[20466], num_nodes=14541)

Data(edge_index=[2, 17535], edge_type=[17535], num_nodes=14541)